# Chapter 6: Advanced Statistical Methods

## Bootstrap Estimator

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

PopData = list()

random.seed(7)

for i in range(1000):
    DataElem = 50 * random.random()
    PopData.append(DataElem)

PopSample = random.choices(PopData, k=100)

PopSampleMean = list()
for i in range(10000):
    SampleI = random.choices(PopData, k=100)
    PopSampleMean.append(np.mean(SampleI))

plt.hist(PopSampleMean)
plt.show()

MeanPopSampleMean = np.mean(PopSampleMean)
print('The mean of the Bootstrap estimator is ', MeanPopSampleMean)

MeanPopData = np.mean(PopData)
print('The mean of the population is ', MeanPopData)

MeanPopSample = np.mean(PopSample)
print('The mean of the simple random sample is ', MeanPopSample)

## Bootstrap Regression

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

x = np.linspace(0, 1, 100)
y = x + (np.random.rand(len(x)))

for i in range(30):
    x = np.append(x, np.random.choice(x))
    y = np.append(y, np.random.choice(y))

x = x.reshape(-1, 1)
y = y.reshape(-1, 1)

reg_model = LinearRegression().fit(x, y)

r_sq = reg_model.score(x, y)
print(f'R squared = {r_sq}')

alpha = float(reg_model.coef_[0])
print(f'slope: {reg_model.coef_}')
beta = float(reg_model.intercept_[0])
print(f'intercept: {reg_model.intercept_}')

y_pred = reg_model.predict(x)

plt.scatter(x, y)
plt.plot(x, y_pred, linewidth=2)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

boot_slopes = []
boot_interc = []
r_sqs = []
n_boots = 500
num_sample = len(x)
data = pd.DataFrame({'x': x[:,0], 'y': y[:,0]})

plt.figure()
for k in range(n_boots):
    sample = data.sample(n=num_sample, replace=True)
    x_temp = sample['x'].values.reshape(-1, 1)
    y_temp = sample['y'].values.reshape(-1, 1)
    reg_model = LinearRegression().fit(x_temp, y_temp)
    r_sqs_temp = reg_model.score(x_temp, y_temp)
    r_sqs.append(r_sqs_temp)
    boot_interc.append(float(reg_model.intercept_[0]))
    boot_slopes.append(float(reg_model.coef_[0]))
    y_pred_temp = reg_model.predict(x_temp)
    plt.plot(x_temp, y_pred_temp, color='grey', alpha=0.2)

plt.scatter(x, y)
plt.plot(x, y_pred, linewidth=2)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

sns.histplot(data=boot_slopes, kde=True)
plt.show()
sns.histplot(data=boot_interc, kde=True)
plt.show()

plt.plot(r_sqs)

max_r_sq = max(r_sqs)
print(f'Max R squared = {max_r_sq}')

pos_max_r_sq = r_sqs.index(max(r_sqs))
print(f'Boot of the best Regression model = {pos_max_r_sq}')

max_slope = boot_slopes[pos_max_r_sq]
print(f'Slope of the best Regression model = {max_slope}')

max_interc = boot_interc[pos_max_r_sq]
print(f'Intercept of the best Regression model = {max_interc}')

## Jackknife Estimator

In [ ]:
import random
import statistics
import matplotlib.pyplot as plt

PopData = list()

random.seed(5)

for i in range(100):
    DataElem = 10 * random.random()
    PopData.append(DataElem)

def CVCalc(Dat):
    CVCalc = statistics.stdev(Dat) / statistics.mean(Dat)
    return CVCalc

CVPopData = CVCalc(PopData)
print(CVPopData)

N = len(PopData)
JackVal = list()
PseudoVal = list()
for i in range(N-1):
    JackVal.append(0)
for i in range(N):
    PseudoVal.append(0)

for i in range(N):
    for j in range(N):
        if(j < i):
            JackVal[j] = PopData[j]
        else:
            if(j > i):
                JackVal[j-1] = PopData[j]
    PseudoVal[i] = N * CVCalc(PopData) - (N-1) * CVCalc(JackVal)

plt.hist(PseudoVal)
plt.show()

MeanPseudoVal = statistics.mean(PseudoVal)
print(MeanPseudoVal)
VariancePseudoVal = statistics.variance(PseudoVal)
print(VariancePseudoVal)
VarJack = statistics.variance(PseudoVal) / N
print(VarJack)

## K-Fold Cross Validation

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

StartedData = np.arange(10, 110, 10)
print(StartedData)

kfold = KFold(5, True, 1)

for TrainData, TestData in kfold.split(StartedData):
    print('Train Data :', StartedData[TrainData], 'Test Data :', StartedData[TestData])

## Permutation Test

In [ ]:
from sklearn.datasets import load_iris
import numpy as np
from sklearn import tree
from sklearn.model_selection import permutation_test_score
import matplotlib.pyplot as plt
import seaborn as sns

data = load_iris()
X = data.data
y = data.target

np.random.seed(0)
X_nc_data = np.random.normal(size=(len(X), 4))

clf = tree.DecisionTreeClassifier(random_state=1)

p_test_iris = permutation_test_score(
    clf, X, y, scoring='accuracy', n_permutations=1000
)

print(f'Score of iris flower classification = {p_test_iris[0]}')
print(f'P_value of permutation test for iris dataset = {p_test_iris[2]}')

p_test_nc_data = permutation_test_score(
    clf, X_nc_data, y, scoring='accuracy', n_permutations=1000
)

print(f'Score of no-correletd data classification = {p_test_nc_data[0]}')
print(f'P_value of permutation test for no-correletd dataset = {p_test_nc_data[2]}')

pbox1 = sns.histplot(data=p_test_iris[1], kde=True)
plt.axvline(p_test_iris[0], linestyle='-', color='r')
plt.axvline(p_test_iris[2], linestyle='--', color='b')
pbox1.set(xlim=(0,1))
plt.show()

pbox2 = sns.histplot(data=p_test_nc_data[1], kde=True)
plt.axvline(p_test_nc_data[0], color='r', linestyle='-')
plt.axvline(p_test_nc_data[2], color='b', linestyle='--')
pbox2.set(xlim=(0,1))
plt.show()